# Pandas reference

Quick reference on getting common data processing tasks done with Pandas.

## Setup

### Import libraries

In [1]:
import numpy as np
import pandas as pd

### Show more data in dataframes

In [2]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 200

### Set floating point precision

In [3]:
pd.options.display.precision = 3

## Datasets

### USD exchange rates (yearly averages)

Data from https://www.irs.gov/individuals/international-taxpayers/yearly-average-currency-exchange-rates:

In [4]:
def usd_exchange_rates_df():
    return pd.DataFrame(
        columns=("Year", "Currency", "Currency/USD", "USD/Currency"),
        data=[
            [pd.to_datetime("2016-12-31"), "EUR", 1.064, 0.940],
            [pd.to_datetime("2017-12-31"), "EUR", 1.083, 0.923],
            [pd.to_datetime("2018-12-31"), "EUR", 1.179, 0.848],
            [pd.to_datetime("2019-12-31"), "EUR", 1.120, 0.893],
            [pd.to_datetime("2020-12-31"), "EUR", 1.140, 0.877],
            [pd.to_datetime("2016-12-31"), "GBP", 1.299, 0.770],
            [pd.to_datetime("2017-12-31"), "GBP", 1.238, 0.808],
            [pd.to_datetime("2018-12-31"), "GBP", 1.333, 0.750],
            [pd.to_datetime("2019-12-31"), "GBP", 1.276, 0.784],
            [pd.to_datetime("2020-12-31"), "GBP", 1.284, 0.779],
        ]
    )

## Series, dataframes and indexes

The basic pandas data type is a series which is a single list of data points:

In [5]:
series = pd.Series(["a", "b", "c", "d"])
series

0    a
1    b
2    c
3    d
dtype: object

The data points have an associated index: a set of labels for the data points, displayed in the left column of the output above. The default index consists simply of the position of the point in the series. Here is a series with an explicit index:

In [6]:
series = pd.Series(["e", "f", "g", "h"], index=["a", "b", "c", "d"])
series

a    e
b    f
c    g
d    h
dtype: object

The elements can be accessed using the index values via the `loc[]` method:

In [7]:
series = pd.Series(["e", "f", "g", "h"], index=["a", "b", "c", "d"])
series.loc["a"]

'e'

Selection by position in the list of data points is always possible using `iloc[]` regardless of what the index is:

In [8]:
series = pd.Series(["e", "f", "g", "h"], index=["a", "b", "c", "d"])
series.iloc[0]

'e'

The key in the index can consists of multiple values, in which case the index is called a *multi-index*. Each value in the tuple is called a *level* of the index and each level can optionally have a name:

In [9]:
index = pd.MultiIndex.from_tuples([("a", "a"), ("a", "b"), ("b", "a"), ("b", "b")], names=["l1", "l2"])
series = pd.Series(["e", "f", "g", "h"], index=index)
series

l1  l2
a   a     e
    b     f
b   a     g
    b     h
dtype: object

A dataframe is a collection of series and it has two indexes: a row index, mapping a row key to a row, and a column index, mapping a column key to a series.

## Selecting rows and columns

### Select with []

Select rows with `[]`:

In [10]:
df = usd_exchange_rates_df()
df[df["Currency"] == "EUR"]

,Year,Currency,Currency/USD,USD/Currency
0,2016-12-31,EUR,1.064,0.940
1,2017-12-31,EUR,1.083,0.923
2,2018-12-31,EUR,1.179,0.848
3,2019-12-31,EUR,1.120,0.893
4,2020-12-31,EUR,1.140,0.877


`[]` will accept a callable as argument for cases where a reference to the dataframe is not available, for example when chaining method calls on the dataframe:

In [11]:
df = usd_exchange_rates_df()
df[lambda df: df["Year"] >= pd.to_datetime("2018-12-31")][lambda df: df["Currency"] == "EUR"]

,Year,Currency,Currency/USD,USD/Currency
2,2018-12-31,EUR,1.179,0.848
3,2019-12-31,EUR,1.120,0.893
4,2020-12-31,EUR,1.140,0.877


Select a single column as a `pd.Series` with `[]`:

In [12]:
df = usd_exchange_rates_df()
df["Currency/USD"]

0    1.064
1    1.083
2    1.179
3    1.120
4    1.140
5    1.299
6    1.238
7    1.333
8    1.276
9    1.284
Name: Currency/USD, dtype: float64

Select one or more columns as a `pd.DataFrame` by passing a list to `[]`:

In [13]:
df = usd_exchange_rates_df()
df[["Currency/USD"]]

,Currency/USD
0,1.064
1,1.083
2,1.179
3,1.120
4,1.140
5,1.299
6,1.238
7,1.333
8,1.276
9,1.284


Selection of rows and of columns can be combined:

In [14]:
df = usd_exchange_rates_df()
df[df["Currency"] == "EUR"]["Currency/USD"]

0    1.064
1    1.083
2    1.179
3    1.120
4    1.140
Name: Currency/USD, dtype: float64

Note that chaining `[]` does not work for the purpose of modifying or inserting data:

In [15]:
df = usd_exchange_rates_df()
df[df["Currency"] == "EUR"]["Currency/USD"] = 5

/tmp/ipykernel_792315/3121943903.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df["Currency"] == "EUR"]["Currency/USD"] = 5


`df[][]=` translates to a `df.__getitem__()` call on the data frame and then a `.__setitem__()` call on the resulting object. The problem is that the `df.__getitem__()` call might return either a view or a copy of the dataframe, so the dataframe might or might not be modified.

Instead, `df.loc[]` can be used to select rows and columns at the same time. `df.loc[]` will return a view or a copy just like `df[]`, but `df.loc[]=` is just a single method call on the `loc` attribute of the original dataframe, free of the ambiguity of `[][]=`, so that it will always correctly modify the dataframe.

### Select with loc[]

Select rows:

In [16]:
df = usd_exchange_rates_df()
df.loc[df["Currency"] == "EUR"]

,Year,Currency,Currency/USD,USD/Currency
0,2016-12-31,EUR,1.064,0.940
1,2017-12-31,EUR,1.083,0.923
2,2018-12-31,EUR,1.179,0.848
3,2019-12-31,EUR,1.120,0.893
4,2020-12-31,EUR,1.140,0.877


`loc[]` will accept a callable as argument for cases where a reference to the dataframe is not available, for example when chaining method calls on the dataframe:

In [17]:
df = usd_exchange_rates_df()
(df.loc[lambda df: df["Year"] >= pd.to_datetime("2018-12-31")]
 .loc[lambda df: df["Currency"] == "EUR"])

,Year,Currency,Currency/USD,USD/Currency
2,2018-12-31,EUR,1.179,0.848
3,2019-12-31,EUR,1.120,0.893
4,2020-12-31,EUR,1.140,0.877


Select a single column as a `pd.Series`:

In [18]:
df = usd_exchange_rates_df()
df.loc[:, "Currency/USD"]

0    1.064
1    1.083
2    1.179
3    1.120
4    1.140
5    1.299
6    1.238
7    1.333
8    1.276
9    1.284
Name: Currency/USD, dtype: float64

Select one or more columns as a `pd.DataFrame`:

In [19]:
df = usd_exchange_rates_df()
df.loc[:, ["Currency/USD"]]

,Currency/USD
0,1.064
1,1.083
2,1.179
3,1.120
4,1.140
5,1.299
6,1.238
7,1.333
8,1.276
9,1.284


Modify a subpart of a dataframe:

In [20]:
df = usd_exchange_rates_df()
df.loc[df["Currency"] == "EUR", "Currency/USD"] = 2
df.loc[df["Currency"] == "GBP", "USD/Currency"] = 0.5
df

,Year,Currency,Currency/USD,USD/Currency
0,2016-12-31,EUR,2.000,0.940
1,2017-12-31,EUR,2.000,0.923
2,2018-12-31,EUR,2.000,0.848
3,2019-12-31,EUR,2.000,0.893
4,2020-12-31,EUR,2.000,0.877
5,2016-12-31,GBP,1.299,0.500
6,2017-12-31,GBP,1.238,0.500
7,2018-12-31,GBP,1.333,0.500
8,2019-12-31,GBP,1.276,0.500
9,2020-12-31,GBP,1.284,0.500


### Select with loc[] and a multi-index

When using `loc[]` with multi-index both the row and column filter need to be passed as arguments:

In [21]:
df = usd_exchange_rates_df().set_index(["Currency", "Year"])
df.loc[("EUR", pd.to_datetime("2017-12-31")), :]

Currency/USD    1.083
USD/Currency    0.923
Name: (EUR, 2017-12-31 00:00:00), dtype: float64

The elements of the tuple can be lists or `slice()` objects:

In [22]:
df = usd_exchange_rates_df().set_index(["Currency", "Year"])
df.loc[(["EUR", "GBP"], slice(pd.to_datetime("2017-12-31"), pd.to_datetime("2019-12-31"))), :]

Currency/USD  USD/Currency
Currency Year                                  
EUR      2017-12-31         1.083         0.923
         2018-12-31         1.179         0.848
         2019-12-31         1.120         0.893
GBP      2017-12-31         1.238         0.808
         2018-12-31         1.333         0.750
         2019-12-31         1.276         0.784

To filter only on first N levels of the multi-index, simply omit the criteria for all remaining levels:

In [23]:
df = usd_exchange_rates_df().set_index(["Currency", "Year"])
df.loc["EUR", :]

,Currency/USD,USD/Currency
Year,,
2016-12-31,1.064,0.940
2017-12-31,1.083,0.923
2018-12-31,1.179,0.848
2019-12-31,1.120,0.893
2020-12-31,1.140,0.877


To filter only on middle or last N levels of the multi-index, use `slice(None)` to select everything at the upper levels (`slice(None)` is just the equivalent of `:`, which can not be used as an element in a tuple):

In [24]:
df = usd_exchange_rates_df().set_index(["Currency", "Year"])
df.loc[(slice(None), pd.to_datetime("2017-12-31")), :]

,,Currency/USD,USD/Currency
Currency,Year,,
EUR,2017-12-31,1.083,0.923
GBP,2017-12-31,1.238,0.808


### Boolean masks for [] and loc[]

Boolean masks can be formed with `&`, `|` and `~` (negation) and passed to `[]` and to `loc[]`. Conditions have to be enclosed in parenthesis since `&` and `|` have higher priority in Python than operators like `>=`:

In [25]:
df = usd_exchange_rates_df()
df[(df["Year"] >= pd.to_datetime("2018-12-31")) &
   (df["Year"] <= pd.to_datetime("2020-12-31"))]

,Year,Currency,Currency/USD,USD/Currency
2,2018-12-31,EUR,1.179,0.848
3,2019-12-31,EUR,1.120,0.893
4,2020-12-31,EUR,1.140,0.877
7,2018-12-31,GBP,1.333,0.750
8,2019-12-31,GBP,1.276,0.784
9,2020-12-31,GBP,1.284,0.779


The condition inside `[]` translates to a boolean vector:

In [26]:
df = usd_exchange_rates_df()
((df["Year"] >= pd.to_datetime("2018-12-31")) &
 (df["Year"] <= pd.to_datetime("2020-12-31")))

0    False
1    False
2     True
3     True
4     True
5    False
6    False
7     True
8     True
9     True
Name: Year, dtype: bool

Use `isin()` series method for subset selection:

In [27]:
df = usd_exchange_rates_df()
df[df["Year"].isin([
    pd.to_datetime("2018-12-31"),
    pd.to_datetime("2019-12-31"),
    pd.to_datetime("2020-12-31")
])]

,Year,Currency,Currency/USD,USD/Currency
2,2018-12-31,EUR,1.179,0.848
3,2019-12-31,EUR,1.120,0.893
4,2020-12-31,EUR,1.140,0.877
7,2018-12-31,GBP,1.333,0.750
8,2019-12-31,GBP,1.276,0.784
9,2020-12-31,GBP,1.284,0.779


## Grouping

### Reduce directly using DataFrameGroupBy/SeriesGroupBy methods

`df.groupby()` returns a `DataFrameGroupBy` that supports calls like `mean()`, `std()`, `min()`, `max()`, etc.:

In [28]:
df = usd_exchange_rates_df()
df.groupby("Currency")[["Currency/USD", "USD/Currency"]].mean()

,Currency/USD,USD/Currency
Currency,,
EUR,1.117,0.896
GBP,1.286,0.778


You can compute a compound expression for each group without repeating the `groupby()` step by using `pipe()`:

In [29]:
df = usd_exchange_rates_df()
(
    df.groupby("Currency")[["Currency/USD", "USD/Currency"]]
    .pipe(lambda currency: currency.quantile(0.75) - currency.quantile(0.25))
)

,Currency/USD,USD/Currency
Currency,,
EUR,0.057,0.046
GBP,0.023,0.014


### Reduce group-by-group series-by-series with agg()

`df.groupby().agg(func)` will call `func(series)` once for each series of every group.

`func` should return a scalar.

In [30]:
df = usd_exchange_rates_df()
df.groupby("Currency").agg(np.mean)

,Year,Currency/USD,USD/Currency
Currency,,,
EUR,2018-12-31 04:48:00,1.117,0.896
GBP,2018-12-31 04:48:00,1.286,0.778


Multiple aggregations can be specified:

In [31]:
df = usd_exchange_rates_df()
df.groupby("Currency")[["Currency/USD", "USD/Currency"]].agg([np.mean, np.var])

Currency/USD        USD/Currency           
                 mean    var         mean        var
Currency                                            
EUR             1.117  0.002        0.896  1.335e-03
GBP             1.286  0.001        0.778  4.462e-04

Use keyword arguments to rename the resulting columns:

In [32]:
df = usd_exchange_rates_df()
df.groupby("Currency")[["Currency/USD", "USD/Currency"]].agg(
    avg_cur2usd=("Currency/USD", np.mean),
    avg_usd2cur=("USD/Currency", np.mean),
)

,avg_cur2usd,avg_usd2cur
Currency,,
EUR,1.117,0.896
GBP,1.286,0.778


The last type of agg() aggregation has slightly different syntax when dealing with a single series:

In [33]:
df = usd_exchange_rates_df()
df.groupby("Currency")["Currency/USD"].agg(average=np.mean)

,average
Currency,
EUR,1.117
GBP,1.286


### Transform rows one-by-one with transform()

`df.groupby().transform(func)` will call `func(series_in_group)` once for each series in each group. In contrast to `apply()`, the result of `transform()` is of the same dimensions as the original dataframe.

`func(series_in_group)` should either return a series of the same dimensions as `series_in_group` or a scalar, in which case pandas will take care of making a series of length `len(series_in_group)` out of it.

In [34]:
df = usd_exchange_rates_df()
df.groupby("Currency")[["Currency/USD", "USD/Currency"]].transform(lambda df: df.mean())

,Currency/USD,USD/Currency
0,1.117,0.896
1,1.117,0.896
2,1.117,0.896
3,1.117,0.896
4,1.117,0.896
5,1.286,0.778
6,1.286,0.778
7,1.286,0.778
8,1.286,0.778
9,1.286,0.778


### Reduce group-by-group with apply()

`df.groupby().apply(func)` will call `func(group)` once for each group, where `group` is a dataframe containing the rows within each group. The form of the result depends of the return type of `func`.

**Case 1:** `func` returns a scalar - the result of `apply(func)` is a series indexed by the group key:

In [35]:
df.groupby("Currency")[["Currency/USD"]].apply(lambda df: np.mean(df.values))

Currency
EUR    1.117
GBP    1.286
dtype: float64

**Case 2:** `func` returns a series - the result of `apply(func)` is a dataframe indexed by the group key, with columns given by the index of the returned series:

In [36]:
df = usd_exchange_rates_df()
df.groupby("Currency")[["Currency/USD", "USD/Currency"]].apply(lambda df: df.mean())

,Currency/USD,USD/Currency
Currency,,
EUR,1.117,0.896
GBP,1.286,0.778


**Case 3:** `func` returns a dataframe - the result of `apply(func)` is a dataframe with a multi-index and with same columns as the dataframe returned by `func`. The multi-index consists of a group key level concatenated with levels of the index of the dataframes returned by `func`:

In [37]:
df = usd_exchange_rates_df()
df.groupby("Currency").apply(lambda df: df.drop(columns=["Currency"]).set_index(["Year"]).rolling(3).mean().dropna())

Currency/USD  USD/Currency
Currency Year                                  
EUR      2018-12-31         1.109         0.904
         2019-12-31         1.127         0.888
         2020-12-31         1.146         0.873
GBP      2018-12-31         1.290         0.776
         2019-12-31         1.282         0.781
         2020-12-31         1.298         0.771

## Pivoting and unpivoting

The USD exchange rates dataframe is neither in fully long format nor in fully wide format: there is one row per each year+currency pair, but there are two different "observations" stored in two columns: "Currency/USD" and "USD/Currency".

Thus the dataframe can be both:
- pivoted (widened) so that each currency becomes a separate column
- unpivoted (lengthened) so that each row is split into two and there is a "Direction" column equal to either "Currency/USD" or "USD/Currency" and a value column with the actual rate

### Pivot with multi-index and unstack()

In [38]:
df = usd_exchange_rates_df().set_index(["Currency", "Year"])
df.unstack(level=0)

Currency/USD        USD/Currency       
Currency            EUR    GBP          EUR    GBP
Year                                              
2016-12-31        1.064  1.299        0.940  0.770
2017-12-31        1.083  1.238        0.923  0.808
2018-12-31        1.179  1.333        0.848  0.750
2019-12-31        1.120  1.276        0.893  0.784
2020-12-31        1.140  1.284        0.877  0.779

### Unpivot with multi-index and stack()

In [39]:
df = usd_exchange_rates_df().set_index(["Currency", "Year"])
df.stack().rename_axis(index={None: "Direction"})

Currency  Year        Direction   
EUR       2016-12-31  Currency/USD    1.064
                      USD/Currency    0.940
          2017-12-31  Currency/USD    1.083
                      USD/Currency    0.923
          2018-12-31  Currency/USD    1.179
                      USD/Currency    0.848
          2019-12-31  Currency/USD    1.120
                      USD/Currency    0.893
          2020-12-31  Currency/USD    1.140
                      USD/Currency    0.877
GBP       2016-12-31  Currency/USD    1.299
                      USD/Currency    0.770
          2017-12-31  Currency/USD    1.238
                      USD/Currency    0.808
          2018-12-31  Currency/USD    1.333
                      USD/Currency    0.750
          2019-12-31  Currency/USD    1.276
                      USD/Currency    0.784
          2020-12-31  Currency/USD    1.284
                      USD/Currency    0.779
dtype: float64

### Pivot with pivot()

In [40]:
df = usd_exchange_rates_df()
df.pivot(index=["Year"], columns=["Currency"], values=["Currency/USD", "USD/Currency"])

Currency/USD        USD/Currency       
Currency            EUR    GBP          EUR    GBP
Year                                              
2016-12-31        1.064  1.299        0.940  0.770
2017-12-31        1.083  1.238        0.923  0.808
2018-12-31        1.179  1.333        0.848  0.750
2019-12-31        1.120  1.276        0.893  0.784
2020-12-31        1.140  1.284        0.877  0.779

### Unpivot with melt()

In [41]:
df = usd_exchange_rates_df()
df.melt(id_vars=["Year", "Currency"], var_name="Direction", value_name="Value")

,Year,Currency,Direction,Value
0,2016-12-31,EUR,Currency/USD,1.064
1,2017-12-31,EUR,Currency/USD,1.083
2,2018-12-31,EUR,Currency/USD,1.179
3,2019-12-31,EUR,Currency/USD,1.120
4,2020-12-31,EUR,Currency/USD,1.140
5,2016-12-31,GBP,Currency/USD,1.299
6,2017-12-31,GBP,Currency/USD,1.238
7,2018-12-31,GBP,Currency/USD,1.333
8,2019-12-31,GBP,Currency/USD,1.276
9,2020-12-31,GBP,Currency/USD,1.284
